In [29]:
from sklearn.datasets import load_breast_cancer
import numpy as np
import collections


In [9]:
! pip install knnor

  Created wheel for knnor: filename=knnor-0.0.1-py3-none-any.whl size=6223 sha256=afc39584cf364217a55755bced5b673f3171a263e8ba53c8532d49e57e351356
  Stored in directory: /Users/amirulislam/Library/Caches/pip/wheels/0f/64/aa/5fb1da90dd286debb4fed4b5c347916d6a488e166adf004a88
Successfully built knnor
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.6/bin/python3.6 -m pip install --upgrade pip' command.


In [17]:
from knnor import data_augment

In [5]:
dataset = load_breast_cancer()

(unique, counts) = np.unique(dataset['target'], return_counts=True)

print('Unique values of the target variable', unique)
print('Counts of the target variable :', counts)

Unique values of the target variable [0 1]
Counts of the target variable : [212 357]


In [32]:
X=dataset["data"]
y=dataset["target"]

print("Original shape=",X.shape,y.shape)
elements_count = collections.Counter(y)
# printing the element and the frequency
print("Original distribution:")
for key, value in elements_count.items():
    print(f"{key}: {value}")

Original shape= (569, 30) (569,)
Original distribution:
0: 212
1: 357


### Augmentation without any parameters

The algorithm calculates the parameters depending on the data

Final result will give an equal number of minority and majority data points

In [33]:
knnor=data_augment.KNNOR()
X_new,y_new=knnor.fit_resample(X,y)
print("Shape after augmentation",X_new.shape,y_new.shape)
elements_count = collections.Counter(y_new)
# printing the element and the frequency
print("Final distribution:")
for key, value in elements_count.items():
    print(f"{key}: {value}")


357 212 (569,)
357 212 (569,)
Shape after augmentation (714, 30) (714,)
Final distribution:
0: 357
1: 357


In [34]:
X_new,y_new=knnor.fit_resample(X,y,
                              num_neighbors=10, # the number of neighbors that will be used for generation of each artificial point
                              max_dist_point=0.01, # the maximum distance at which the new point will be placed
                              proportion_minority=0.3, # proportion of the minority population that will be used to generate the artificial point
                              final_proportion=2 # final number of minority datapoints
                               # example, if num majority =15 and num minority =5, 
#                                putting final_proportion as 1 will add 10 artificial minority points
                              )
print("Shape after augmentation",X_new.shape,y_new.shape)
elements_count = collections.Counter(y_new)
# printing the element and the frequency
print("Final distribution:")
for key, value in elements_count.items():
    print(f"{key}: {value}")


357 212 (569,)
Shape after augmentation (1071, 30) (1071,)
Final distribution:
0: 714
1: 357
